<a href="https://colab.research.google.com/github/Leena811/civic_eye/blob/main/final_civic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 1.1 Enable GPU: Runtime -> Change runtime type -> GPU
# 1.2 Run this cell to mount Drive and check GPU + TF
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
DATA_DIR = '/content/drive/MyDrive/civic_eye_dataset_new'
# The four folders (exact names you provided):
classes = ['garbage_images','potholes_images','sewage_drainage_images','street_light_images']


In [7]:
import os, glob
for c in classes:
    files = []
    for ext in ('*.jpg','*.jpeg','*.png'):
        files += glob.glob(os.path.join(DATA_DIR, c, ext))
    print(f"{c}: {len(files)} images")


garbage_images: 270 images
potholes_images: 250 images
sewage_drainage_images: 227 images
street_light_images: 254 images


In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
import glob

filepaths = []
labels = []
for idx, c in enumerate(classes):
    for ext in ('*.jpg','*.jpeg','*.png'):
        for f in glob.glob(os.path.join(DATA_DIR, c, ext)):
            filepaths.append(f)
            labels.append(idx)

filepaths = np.array(filepaths)
labels = np.array(labels)

# 70% train, 30% temp -> then split temp into val & test (50/50 -> 15% each)
X_train, X_temp, y_train, y_temp = train_test_split(
    filepaths, labels, stratify=labels, test_size=0.30, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)

print("train, val, test sizes:", len(X_train), len(X_val), len(X_test))



train, val, test sizes: 700 150 151


In [10]:
import tensorflow as tf
AUTOTUNE = tf.data.AUTOTUNE
IMG_SIZE = (224, 224)     # works well with EfficientNetB0
BATCH_SIZE = 32

# augmentation (used only for training)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.12),
    tf.keras.layers.RandomZoom(0.12),
    tf.keras.layers.RandomContrast(0.08),
])

def decode_resize(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # handles jpg/jpeg/png
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32)
    return image

# EfficientNet expects its preprocess_input (scales as needed)
preprocess = tf.keras.applications.efficientnet.preprocess_input

def process_train(path, label):
    image = decode_resize(path)
    image = data_augmentation(image)
    image = preprocess(image)   # now it's ready for the pretrained model
    return image, label

def process_eval(path, label):
    image = decode_resize(path)
    image = preprocess(image)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = (train_ds.shuffle(len(X_train))
                    .map(lambda x,y: process_train(x,y), num_parallel_calls=AUTOTUNE)
                    .batch(BATCH_SIZE)
                    .prefetch(AUTOTUNE))

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).map(process_eval, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(process_eval, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)


In [11]:
from sklearn.utils import class_weight
class_weights_arr = class_weight.compute_class_weight('balanced',
                                                     classes=np.unique(y_train),
                                                     y=y_train)
class_weights = {i: w for i, w in enumerate(class_weights_arr)}
print("class_weights:", class_weights)


class_weights: {0: np.float64(0.9259259259259259), 1: np.float64(1.0), 2: np.float64(1.10062893081761), 3: np.float64(0.9887005649717514)}


In [12]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False, weights='imagenet', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False   # freeze for head training

inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(len(classes), activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │         5,124 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,054,695 (15.47 MB)

 Trainable params: 5,124 (20.02 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [14]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/civic_eye_model_best.keras',
                                       save_best_only=True, monitor='val_loss'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
]

initial_epochs = 10
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds,
                    class_weight=class_weights,
                    callbacks=callbacks)



Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 131s 6s/step - accuracy: 0.4389 - loss: 1.1911 - val_accuracy: 0.8800 - val_loss: 0.5769 - learning_rate: 0.0010
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.8689 - loss: 0.5406 - val_accuracy: 0.9200 - val_loss: 0.3415 - learning_rate: 0.0010
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.9188 - loss: 0.3491 - val_accuracy: 0.9400 - val_loss: 0.2588 - learning_rate: 0.0010
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.9520 - loss: 0.2414 - val_accuracy: 0.9600 - val_loss: 0.2140 - learning_rate: 0.0010
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - accuracy: 0.9575 - loss: 0.1943 - val_accuracy: 0.9533 - val_loss: 0.1809 - learning_rate: 0.0010
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 84s 4s/step - accuracy: 0.9489 - loss: 0.1853 - val_accuracy: 0.9667 - val_loss: 0.1598 - learning_rate: 0.0010
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 90s 4s/step - accuracy: 0.9812 - loss: 0.1519 - val_accuracy

In [16]:
# Unfreeze last N layers of base_model for fine-tuning
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 30  # you can experiment with this number
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

fine_tune_epochs = 15
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds,
                         class_weight=class_weights,
                         callbacks=callbacks)


Epoch 10/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 125s 4s/step - accuracy: 0.9277 - loss: 0.3217 - val_accuracy: 0.9733 - val_loss: 0.1306 - learning_rate: 1.0000e-05
Epoch 11/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 94s 4s/step - accuracy: 0.9517 - loss: 0.2961 - val_accuracy: 0.9733 - val_loss: 0.1378 - learning_rate: 1.0000e-05
Epoch 12/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 94s 4s/step - accuracy: 0.9477 - loss: 0.2872 - val_accuracy: 0.9733 - val_loss: 0.1442 - learning_rate: 1.0000e-05
Epoch 13/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 93s 4s/step - accuracy: 0.9465 - loss: 0.2764 - val_accuracy: 0.9733 - val_loss: 0.1491 - learning_rate: 1.0000e-05
Epoch 14/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 93s 4s/step - accuracy: 0.9348 - loss: 0.2772 - val_accuracy: 0.9733 - val_loss: 0.1551 - learning_rate: 5.0000e-06
Epoch 15/25
22/22 ━━━━━━━━━━━━━━━━━━━━ 93s 4s/step - accuracy: 0.9487 - loss: 0.2449 - val_accuracy: 0.9733 - val_loss: 0.1603 - learning_rate: 5.0000e-06


In [17]:
# 9.1 model.evaluate
loss, acc = model.evaluate(test_ds)
print("Test loss:", loss, "Test accuracy:", acc)

# 9.2 predictions -> classification report + confusion matrix
import numpy as np
y_true = np.concatenate([y.numpy() for x,y in test_ds], axis=0)
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred, target_names=classes))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))


5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 0.9777 - loss: 0.1212
Test loss: 0.12023565918207169 Test accuracy: 0.9668874144554138
5/5 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step
                        precision    recall  f1-score   support

        garbage_images       1.00      0.95      0.97        41
       potholes_images       0.97      0.97      0.97        37
sewage_drainage_images       0.92      0.97      0.94        34
   street_light_images       0.97      0.97      0.97        39

              accuracy                           0.97       151
             macro avg       0.97      0.97      0.97       151
          weighted avg       0.97      0.97      0.97       151

Confusion matrix:
 [[39  0  2  0]
 [ 0 36  0  1]
 [ 0  1 33  0]
 [ 0  0  1 38]]


In [18]:
# ✅ Save model in .keras format
model_save_path = '/content/drive/MyDrive/civic_eye_model_final.keras'
model.save(model_save_path)
print("Saved model to:", model_save_path)

# (Optional) if you also want .h5
# model.save('/content/drive/MyDrive/civic_eye_model_final.h5')


Saved model to: /content/drive/MyDrive/civic_eye_model_final.keras


In [25]:
from google.colab import files
from PIL import Image
import numpy as np
import tensorflow as tf

# ✅ Upload image(s) using file picker
uploaded = files.upload()

# Mapping labels -> folder -> department
label_to_folder = {i: classes[i] for i in range(len(classes))}
label_to_department = {
    'garbage_images': 'Department of sanitation',
    'potholes_images': 'Department of Road and transport',
    'sewage_drainage_images': 'Department of sewage and drainage',
    'street_light_images': 'Department of street light'
}

def predict_image(path):
    img = Image.open(path).convert('RGB').resize(IMG_SIZE)
    arr = np.array(img).astype(np.float32)
    arr = tf.keras.applications.efficientnet.preprocess_input(arr)
    arr = np.expand_dims(arr, 0)

    probs = model.predict(arr)[0]
    idx = int(np.argmax(probs))
    folder_name = label_to_folder[idx]
    dept = label_to_department[folder_name]
    return {
        'pred_label': folder_name,
        'department': dept,
        'probability': float(probs[idx])
    }

# ✅ Run prediction for each uploaded file
for fn in uploaded.keys():
    result = predict_image(fn)
    print(f"\n📷 File: {fn}")
    print(f"Predicted Class: {result['pred_label']}")
    print(f"Department: {result['department']}")
    print(f"Confidence: {result['probability']*100:.2f}%")


Saving 9.jpg to 9 (1).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step

📷 File: 9 (1).jpg
Predicted Class: potholes_images
Department: Department of Road and transport
Confidence: 98.95%


In [26]:
import os

# Path where you saved your model
model_save_path = '/content/drive/MyDrive/civic_eye_model_final.keras'

# Check if file exists
if os.path.exists(model_save_path):
    print("✅ Model file exists:", model_save_path)
    print("File size (MB):", round(os.path.getsize(model_save_path) / (1024*1024), 2))
else:
    print("❌ Model file not found at:", model_save_path)


✅ Model file exists: /content/drive/MyDrive/civic_eye_model_final.keras
File size (MB): 27.75
